# 1) Extraindo os dados dos arquivos PDF
> Seção introdutória (markdown) que anuncia a etapa de extração de textos a partir de PDFs.

---

## Bloco 1 — Instalações de dependências
**O que faz:** instala bibliotecas para leitura de PDF:

- pypdf — parser de PDFs (lê estrutura, páginas).

- pdfminer.six — extrator de texto de PDF (converte PDF→texto).

**Por quê:** garantir ambiente reprodutível com versões fixadas (evita “quebras” ao longo do tempo).

**Saída:** nenhuma variável; apenas instala pacotes no ambiente.

In [1]:
%pip install pypdf==4.3.1 pdfminer.six==20240706


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Bloco 2 — Varredura e cache de parágrafos por pasta/ano

**O que faz:** percorre a **raiz** com PDFs, filtra por **anos** desejados, extrai parágrafos de cada documento (via `processar_pdf`) e salva/recupera **cache** em `*.pkl` (arquivo binário do pandas) para acelerar novas execuções.

**Parâmetros (principais):**

* `raiz` (caminho da pasta): diretório onde estão os PDFs.
* `min_palavras` (limiar de qualidade): descarta/funde parágrafos muito curtos.
* `anos` (filtro temporal): limita os PDFs pelos anos no nome do arquivo (padrão `{2022, 2023, 2024, 2025}`).
* `cache_dir` (cache em disco): pasta dos `*.pkl`.
* `rebuild_cache` (reprocessar?): se `True`, ignora cache e reconstrói.

**Conceitos rápidos:**

* **Cache**: armazenamento intermediário de resultados para evitar reprocessar tudo a cada execução.
* **YYYYMM**: carimbo de ano+mês (ex.: `202312`).

**Saída:** `DataFrame` com colunas `["doc","dt","pid","texto","n_palavras"]`, ordenado.


---

In [14]:
def processar_raiz(raiz: str,
                   min_palavras: int = 8,
                   anos: set[int] | None = {2022, 2023, 2024, 2025},
                   cache_dir: str = "out/estatisticas_paragrafos",
                   rebuild_cache: bool = False) -> pd.DataFrame:
    raiz = Path(raiz)
    cache = Path(cache_dir)
    cache.mkdir(parents=True, exist_ok=True)

    dfs, vistos = [], set()
    # 1) varredura recursiva em subpastas yyyy
    for pdf in sorted(raiz.rglob("*.pdf")):
        name = pdf.stem  # ex.: "202212_Texto_de_estatisticas_monetarias_e_de_credito"
        # 2) captura robusta do YYYYMM onde quer que esteja no nome
        m = re.search(r'(?<!\d)(\d{6})(?!\d)', name)
        if not m:
            continue
        yyyymm = m.group(1)
        ano = int(yyyymm[:4])
        mes = int(yyyymm[4:6])

        # limita por anos desejados
        if anos is not None and ano not in anos:
            continue
        # evita duplicatas (mesmo YYYYMM em mais de um arquivo)
        if yyyymm in vistos:
            continue

        # 3) cache .pkl por documento
        pkl_path = cache / f"{yyyymm}.pkl"
        if (not rebuild_cache) and pkl_path.exists():
            try:
                df = pd.read_pickle(pkl_path)
            except Exception:
                df = processar_pdf(pdf, min_palavras=min_palavras)
                df.to_pickle(pkl_path)
        else:
            df = processar_pdf(pdf, min_palavras=min_palavras)
            df.to_pickle(pkl_path)

        if not df.empty:
            dfs.append(df)
            vistos.add(yyyymm)

    if not dfs:
        return pd.DataFrame(columns=["doc","dt","pid","texto","n_palavras"])

    out = pd.concat(dfs, ignore_index=True)
    return out.sort_values(["dt","doc","pid"]).reset_index(drop=True)


**O que faz:**

1. Executa a coleta/extração de parágrafos e guarda em `df_paragrafos`.
2. Exporta para `paragrafos.csv`.

**Por quê:** persistir em CSV facilita auditoria e uso em outras ferramentas.

**Saída:** arquivo `paragrafos.csv` no diretório atual.

---

In [9]:
df_paragrafos = processar_raiz("data/estatisticas",
                               min_palavras=8,
                               anos={2022, 2023, 2024, 2025},
                               rebuild_cache=True)    # 1ª rodada: gera o cache de todos
df_paragrafos

df_paragrafos.to_csv(
    "paragrafos.csv",
    index=False,          
    header=True,          
    na_rep="",            # como representar valores ausentes (NaN)
    float_format="%.2f",  
)



# 2.1) Limpando títulos desnecessários

> Seção (markdown) que introduz a **limpeza textual**: remoção de cabeçalhos/legendas, fusão de parágrafos curtos etc.

---
## Bloco 4 — Funções utilitárias de limpeza e fusão


In [10]:
import re
import numpy as np
import pandas as pd

# --- helpers ---
WORD_RE = re.compile(r"\b\w+\b", flags=re.UNICODE)

def count_words(s: str) -> int:
    return len(WORD_RE.findall(s or ""))

def caps_ratio(s: str) -> float:
    toks = re.findall(r"\b[^\W\d_]+\b", s or "", flags=re.UNICODE)
    if not toks: 
        return 0.0
    return sum(t.isupper() for t in toks) / len(toks)

_HEADING_PREFIXES = (
    "nota para a imprensa",
    "sumário","sumario","resumo",
    "apresentação","apresentacao",
    "introdução","introducao",
    "conclusão","conclusoes","conclusao",
)
_CAPTION_HINTS = ("figura","gráfico","grafico","tabela","fonte:")

def is_heading(text: str) -> bool:
    s = (text or "").strip()
    if not s:
        return True
    # "2." ou "2.3." no início
    if re.match(r"^\d+(\.\d+)*\s", s):
        return True
    low = s.lower()
    if low.startswith(_HEADING_PREFIXES):
        return True
    if any(k in low for k in _CAPTION_HINTS):
        return True
    # muito MAIÚSCULO (título), curto e sem pontuação final
    if caps_ratio(s) > 0.6 and len(s.split()) <= 20:
        return True
    if len(s.split()) < 15 and not re.search(r"[.!?;:]\s*$", s):
        return True
    return False

def clean_text_unit(s: str) -> str:
    s = (s or "").replace("\r","")
    s = re.sub(r"[ \t]+", " ", s).strip()
    # remove numeração de seção no início
    s = re.sub(r"^\d+(\.\d+)*\s+", "", s)
    return s

def _fuse_short_paragraphs(df_group: pd.DataFrame, min_words: int = 8, max_merge_span: int = 3) -> pd.DataFrame:
    """Funde parágrafos consecutivos curtos até atingir min_words (no máx. 3 blocos por fusão)."""
    rows, buf, buf_words, buf_span = [], [], 0, 0
    for _, r in df_group.sort_values("pid").iterrows():
        txt = str(r["texto"])
        w = count_words(txt)
        if w >= min_words:
            if buf and buf_words >= min_words:
                rows.append({"texto": " ".join(buf)})
            buf, buf_words, buf_span = [], 0, 0
            rows.append({"texto": txt})
        else:
            buf.append(txt); buf_words += w; buf_span += 1
            if buf_words >= min_words or buf_span >= max_merge_span:
                rows.append({"texto": " ".join(buf)})
                buf, buf_words, buf_span = [], 0, 0
    if buf and buf_words >= min_words:
        rows.append({"texto": " ".join(buf)})
    out = pd.DataFrame(rows)
    out["pid"] = np.arange(1, len(out)+1, dtype=int)
    return out

def limpar_paragrafos(df: pd.DataFrame, min_palavras: int = 8, max_merge_span: int = 3) -> pd.DataFrame:
    """
    Entrada: df_paragrafos com colunas ['doc','dt','pid','texto','n_palavras'].
    Saída: df_limpo com mesmas colunas, mas:
      - headings/legendas removidos
      - parágrafos curtos fundidos
      - mínimo de 8 palavras garantido
      - pid reindexado por (doc, dt)
    """
    # checagem leve
    for c in ["doc","dt","pid","texto","n_palavras"]:
        if c not in df.columns:
            raise ValueError(f"Coluna obrigatória ausente: {c}")

    dfx = df.copy()
    dfx["doc"] = dfx["doc"].astype(str)
    dfx["dt"] = pd.to_datetime(dfx["dt"], errors="coerce")
    dfx["pid"] = pd.to_numeric(dfx["pid"], errors="coerce").fillna(0).astype(int)
    dfx["texto"] = dfx["texto"].astype(str)

    # 1) remove vazios/sem letras
    mask_nonempty = dfx["texto"].str.strip().ne("")
    mask_alpha = dfx["texto"].str.contains(r"[A-Za-zÀ-ÖØ-öø-ÿ]", regex=True)
    dfx = dfx[mask_nonempty & mask_alpha].copy()

    # 2) remove headings/legendas
    dfx = dfx[~dfx["texto"].map(is_heading)].copy()

    # 3) limpeza fina + recount
    dfx["texto"] = dfx["texto"].map(clean_text_unit)
    dfx["n_palavras"] = dfx["texto"].map(count_words).astype(int)

    # 4) fusão de curtas por (doc, dt)
    groups = []
    for (doc, dt), g in dfx.sort_values(["doc","pid"]).groupby(["doc","dt"], sort=False):
        g2 = _fuse_short_paragraphs(g[["pid","texto"]], min_words=min_palavras, max_merge_span=max_merge_span)
        g2["doc"], g2["dt"] = doc, dt
        groups.append(g2)
    if groups:
        dfx = pd.concat(groups, ignore_index=True)
    else:
        # nenhum grupo → retorna vazio com colunas padrão
        return dfx.assign(n_palavras=pd.Series(dtype=int))[["doc","dt","pid","texto","n_palavras"]]

    # 5) recount + mínimo final
    dfx["texto"] = dfx["texto"].map(clean_text_unit)
    dfx["n_palavras"] = dfx["texto"].map(count_words).astype(int)
    dfx = dfx[dfx["n_palavras"] >= min_palavras].copy()

    # 6) ordena e reindexa pid por doc
    dfx = dfx.sort_values(["dt","doc","pid"]).reset_index(drop=True)
    dfx["pid"] = dfx.groupby(["doc","dt"]).cumcount() + 1
    return dfx[["doc","dt","pid","texto","n_palavras"]]


**O que faz (principais utilitários):**

* `count_words` — conta **palavras** (tokens) no texto.
* `caps_ratio` — mede proporção de **palavras em MAIÚSCULAS** (útil para detectar títulos).
* `is_heading` — heurística para filtrar **títulos/legendas** (ex.: começa com numeração “2.3”, contém “Figura/Tabela”, alto `caps_ratio`, sem pontuação final).
* `clean_text_unit` — limpeza fina: remove quebras, espaços excessivos, numeração de seção.
* `_fuse_short_paragraphs` — **funde parágrafos muito curtos** (até atingir `min_words`, limitando `max_merge_span`).
* `limpar_paragrafos` — **pipeline** de limpeza:

  1. remove vazios/sem letras,
  2. remove headings/legendas,
  3. limpeza fina + **reconta palavras**,
  4. funde curtos **por (doc, dt)**,
  5. reforça mínimo de palavras,
  6. **reindexa `pid`** (posição do parágrafo no documento/data).

**Conceitos rápidos:**

* **Heurística**: regra prática que funciona “na maioria dos casos”.
* **Reindexar `pid`**: renumerar a ordem dos parágrafos após limpeza/fusão para manter consistência.

**Saída:** funções prontas para uso; nenhuma tabela imediata.

---

## Bloco 5 — Aplicação da limpeza e amostra

In [11]:
# se você já tem df_paragrafos carregado:
df_paragrafos_limpos = limpar_paragrafos(df_paragrafos, min_palavras=8, max_merge_span=3)

# checagem rápida
print(df_paragrafos.shape, "→", df_paragrafos_limpos.shape)
display(df_paragrafos_limpos.head(10))


(1079, 5) → (793, 5)


,doc,dt,pid,texto,n_palavras
0,202201,2022-01-01,1,"Em 2021, o saldo do crédito ampliado ao setor ...",250
1,202201,2022-01-01,2,"O volume de crédito do SFN alcançou R$4,7 tril...",206
2,202201,2022-01-01,3,"O crédito livre às famílias atingiu R$ 1,5 tri...",56
3,202201,2022-01-01,4,"Em 2021, o crédito direcionado atingiu R$ 1, 9...",166
4,202201,2022-01-01,5,"O Indicador de Custo do Crédito (ICC), que med...",93
5,202201,2022-01-01,6,A taxa média de juros das contratações finaliz...,203
6,202201,2022-01-01,7,"A inadimplência do crédito geral atingiu 2, 3%...",59
7,202201,2022-01-01,8,"3. Agregados monetários Em 2021, a base monetá...",363
8,202201,2022-01-01,9,De acordo com a Política de Revisão das Estatí...,85
9,202201,2022-01-01,10,i) Crédito por setor de atividade econômica : ...,101


**O que faz:** aplica a limpeza completa sobre `df_paragrafos`, checa o **efeito na forma** (linhas/colunas) e exibe **amostra** (primeiras 10 linhas).

**Por quê:** validar que títulos/legendas foram removidos, parágrafos curtos foram fundidos e que a estrutura (`doc`, `dt`, `pid`, `texto`, `n_palavras`) permaneceu correta.

**Saída:** `df_paragrafos_limpos` (dataframe limpo).

# 2) DeB3RTA

> Seção (markdown) que indica o próximo passo do notebook: uso do modelo **DeB3RTa** (variante do DeBERTa) para tarefas de linguagem.
> Dica: aqui normalmente entram **tokenização** (quebrar texto em subpalavras), **preparo de dataset** e **inferência/treino**.

